# OpenPetition.de Recommendation System

The following Jupyter notebook was created during the [DataBBQ "Open Data in Action"](https://www.meetup.com/de-DE/Data-Visualization-RheinMain/events/239007070/), organized by the [DataVizRM Meetup](https://www.meetup.com/de-DE/Data-Visualization-RheinMain/) and hosted by [SAS](https://www.sas.com/de_de/company-information/office-listing.html) in Heidelberg.

This notebook is part of a team effort for the mini-hackathon held during the BBQ. The team members are:
  * Cornelius Schwab
  * Danny Tobisch
  * Dilshod
  * Dirk Toewe

The goal of the hackathon was to improve the [openpetition](https://www.openpetition.de/) platform using the the anonymized user data which was published beforehand. This notebook represents the preprocessing step used to convert the user into a reduced form suited for visualization.

In [1]:
% matplotlib notebook
from pandas import read_csv, DataFrame
from collections import defaultdict, Counter
from itertools import permutations, groupby
import csv, os, numpy as np, pandas as pd
from ipywidgets import FloatProgress
from IPython.display import display

# Reading Raw Data

For our recommendation system we needed to know, for each pair of petitions (p,q), how many signatures they have in common. Our hypothesis was that if two petitions have alot of signatures in common they are closely related and for a new signer of p, q would be a sensible recommendation in that case.

The signature data can be found in *signer.csv*. The file is however to large to read in as [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). Instead the data is streamed using the [csv](https://docs.python.org/2/library/csv.html) module. Unnecessary data is discarded directly.

In [2]:
def signatures():

  filepath = os.path.expanduser('~/Documents/openpetition_data/signer.csv')

  with open(filepath,'r') as file:

    def lines():
      prgrs= FloatProgress(
        description = '0%',
        min=0, max=os.path.getsize(filepath)
      )
      prgrs_val = 0
      display(prgrs)
      for line in file:
        yield line
        prgrs_val += len( line.encode('utf-8') )
        if prgrs_val*100//prgrs.max != prgrs.value*100//prgrs.max:
          prgrs.description = '%3d%%' % (prgrs_val*100//prgrs.max)
          prgrs.value = prgrs_val

    reader = csv.reader(lines(), delimiter=',', quotechar='"')

    head = { field: i for i,field in enumerate(next(reader)) }
    iEmail= head['email']
    iPid  = head['petition_id']
    del head
    emails = []

    def pids():
        
      for row in reader:
        email = row[iEmail]
        if 0 < len(email):
          pid = row[iPid]
          assert pid.startswith('=')
          emails.append(email)
          yield pid[1:]

    pids = np.fromiter( pids(), dtype= np.int64 )
    return emails,pids

signatures = signatures()
signatures = DataFrame({
  'email'      : signatures[0],
  'petition_id': signatures[1]
})

The parsed results are a list of email-petition_id-pairs, each of which represents a person signing for particular petition. We hereby assumed the E-Mail to be a unique identifier for assigner. Anonymous signatures, i.e. without E-Mail were discarded.

In [3]:
signatures

,email,petition_id
0,2578a1e52f55b26e05bfd98466b7c9cab148b52e6b0f1e...,10
1,1bb0967e11d9d231b9f5994ab3ae0011c1cecd1f965728...,10
2,9c6e24e4d7cfe19f73fc3aad91508f1436b2dfd87f1b83...,12
3,c419eeb8de93da93176f9b45d8ad3a47244d728efdaa82...,12
4,fee2f0035d3d077fddcbd4fc8f4f49d33e523ebb5c5237...,12
5,f1bc02adcd84cfd2947e65c1147b0f436548c0da3f72b6...,12
6,35c04b0af8bee2a0a0843aae98b6fa301d99dacad2db8a...,12
7,66815d6f10501fa95d45e60782a96c1cc92b29a86948bd...,12
8,af9e79b7768f65cba3f1703427f4bdf20f3cca9f5aebb6...,12
9,83ae301fe0bf1e99b298dba1ea807f381c1fddbe281881...,12


# Group by E-Mail

The data presentation is not ideal for the purposes of a recommendation system yet. We are not interested in particular signatures but rather in all signatures that two petitions have in common. The first step of processing to group all petition_ids signed by an individual e-mail/user. At the same time we can already filter out all the e-mail only used to sign a single petition as they don't give us any connection between two or more petitions.

In [4]:
multisigner = defaultdict(set)
for row in signatures.itertuples():
  multisigner[row.email].add(row.petition_id)

del signatures # <- save memory

multisigner = {
  k : np.fromiter(v, dtype=np.int64)
  for k,v in multisigner.items()
  if len(v) > 1
}
multisigner

{'6ecb9aaaa145fe91be5a22836535eab08be642e7632e783a90888a85a8175bf9': array([27260, 17550]),
 '7c9c2662e4fc5e436bbfa4192c7a0a37b374df7082d795a66273185b6e9f5978': array([22353,  5106, 12618, 18047]),
 '6f69620598552fa1de2289a6bd181b2f8ced911583dd260bf1f0d02add9302c7': array([24857, 25897]),
 'c16f322b86b90fff6f9971ded101f18b6ac2d712a70d4d4b80835931f718b132': array([ 5704, 13473, 16293, 15190]),
 '2465175fa0c1f38610a7fb71694424bfb735284d863e3c9b8bead8a9a58d233e': array([9289, 9218]),
 '0710f973029b85b17827c148c698a0f401e28a195f2c87e521493afc3a9f4f2b': array([13746, 17181]),
 'b80e761f4ef3eb76598be1a2d88a5bdd0551e35165bd5b99330473b7fb28bb37': array([ 4696, 17155]),
 'ad37e16bb583f50a738378975d634b4251c2a7e2dd991f6a50938f7a244a47ef': array([23172, 17550]),
 'aedf8196bb5ba640224f0d7c1ca630d71dd90667233aa28dd7274ca1180bc310': array([23336, 25050]),
 '55a5ed112cc32dd28085cb270d2c4bf0af33df4befe47f6ae4b9b9ccefc8cc2a': array([15536, 17181]),
 'ae46ff47d470e3db338adf457ca195c900c613294ffb2bd6c6c7

# Filter Outliers

It is interesting to see what is the highest number of petitions signed using the same e-mail. As is turns out there is an e-mail which was used in 2174 petitions which is more than suspicious. Overall there are however only 88 e-mails used in more than 200 petitions. 

In [5]:
bot = max( multisigner.items(), key = lambda kv: len(kv[1]) )
print( len(bot[1]) )
len({
  k : v
  for k,v in multisigner.items()
  if len(v) > 200
})

2159


66

Let us filter out all the users which have signed a hundred petitions of less considering them as "normal" users. As it turns out the remaining sample size is still over 1 million, which is more than reasonable

In [6]:
nonbots = {
  k: v
  for k,v in multisigner.items()
  if len(v) <= 100
}
len(nonbots)

1028282

# Building Graph

In this final step data transformation, we turn the data into a weighted <a href="https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)">undirected graph</a>. For each pair of petitions a user has signed, we count up the weight of the edge between the petitions by 1.

In [7]:
graph = Counter(
  ab
  for v in nonbots.values()
  for ab in permutations(v,2)
)

print( len(graph) )

3716164


Now all we have to is to save the data so that we can visualize it. We utilized [Microsoft Power BI](https://powerbi.microsoft.com/de-de/).

In [8]:
with open( os.path.expanduser('~/Documents/openpetition_data/graph.csv'), 'w') as signer:
  writer = csv.writer(signer, delimiter=',', quotechar='"')
  writer.writerow(['petition_from','petition_to','weight'])
  for (a,b),n in graph.items():
    writer.writerow([a,b,n])

# Reducing to Recommendations

Since the graph potentially contains a large amount of edges, we can go on an remove all but the 5 strongest edges for each petition node. Keep in mind that this results in a *directed graph*.

In [9]:
recommendations = defaultdict(list)

for (a,b),n in graph.items():
  recommendations[a] += [b,n]

nVertices = 5

def rows():
  for a,bn in recommendations.items():
    bn = np.array(bn).reshape(-1,2)
    yield a, bn[ np.lexsort(bn.T)[:-nVertices-1:-1] ]

recommendations = list( rows() )

del rows
recommendations.sort(key = lambda kv: kv[0])
for k,v in recommendations[:+2]:
  print('%d ->\n%s' % (k,v) )
print('...')
for k,v in recommendations[-2:]:
  print('%d ->\n%s' % (k,v) )

12 ->
[[2081   46]
 [2846   44]
 [4031   42]
 [  30   34]
 [4606   32]]
13 ->
[[ 2081    10]
 [ 7047     8]
 [ 6181     8]
 [24766     7]
 [18293     7]]
...
27854 ->
[[18178     1]
 [18047     1]
 [17611     1]
 [14630     1]
 [11114     1]]
27857 ->
[[27166     1]]


We can also incorporate the petition name into the graph.

In [10]:
petitions = read_csv(
  os.path.expanduser('~/Documents/openpetition_data/petition.csv'),
  index_col='petition_id', usecols=['petition_id','title']
)
assert petitions.index.str.startswith('=').all()
petitions.index = pd.to_numeric( petitions.index.str[1:] )
petitions = petitions.loc[np.fromiter( (a for (a,_) in graph), dtype=np.int64 )]

petitions = {
  row[0]: row.title
  for row in petitions.itertuples()
}
petitions

{16384: '"legale Wände" in Kiel // Schaffung von Flächen für Graffiti',
 16386: 'Hebammen, wir brauchen euch!',
 16389: 'LuftschlossfAbrik bleibt!',
 16391: 'Stoppt den Kinderklau,gebt den Müttern ihre Rechte zurück!Lasst unsere  Kinder wieder nach Hause',
 10924: 'Mehr Bewegung für Neu-Isenburg, der Pumptrack soll bleiben!',
 16394: 'Marcel Reif sollte als Kommentator nicht mehr arbeiten dürfen',
 5463: 'Unterstützung für die Neuendorfer Bürger: Die Insel Hiddensee darf kein "zweites Sylt" werden!',
 16396: 'Sofortige Umsetzung von geltendem Recht und bestehender Gesetze (Asylrecht) in Deutschland',
 16397: 'Wir fordern EINE gleichwertige, zentrale und langfristige Unterbringung für den "Brückenhaus e.V."',
 14: 'Reitwege in Schleswig-Holstein',
 16399: 'Winterferien in Baden-Württemberg',
 16401: 'Wir lassen uns unser Wahlrecht nicht nehmen und fordern den Erhalt der Förderschulen Lernen!',
 16402: 'Fall Edathy und Ecclestone: Keinen Freikauf mehr "Im Namen des Volkes" ermöglichen!',

In [11]:
with open( os.path.expanduser('~/Documents/openpetition_data/recommendations.csv'), 'w') as signer:
  writer = csv.writer(signer, delimiter=',', quotechar='"')
  writer.writerow(['petition_from','petition_to','weight','title_from','title_to'])
  for a,bn in recommendations:
    for b,n in bn:
      writer.writerow([
        a,b,n,
        petitions[a],
        petitions[b]
      ])